## ChromaDB 만들고 답변 가지고 오기

---
### Chromadb
- embedding model : 'all-MiniLM-L6-v2'
- Document(page_content = 태그 , metadata = 나머지 정보들)

In [1]:
!pip install chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
   -- ------------------------------------- 1.3/19.8 MB 6.7 MB/s eta 0:00:03
   --------- ------------------------------ 4.5/19.8 MB 11.2 MB/s eta 0:00:02
   ------------- -------------------------- 6.8/19.8 MB 11.3 MB/s eta 0:00:02
   ------------------------ --

In [1]:
import chromadb

client = chromadb.PersistentClient(path='./chromadb_tag_only')

In [2]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
collection = client.get_or_create_collection(name='tour_tag')

In [4]:
# pd 생성 후 태그만 임베딩하기
import pandas as pd

df = pd.read_csv('C:\\Users\\brain\\Desktop\\3차 프로젝트\\GIT\\crawling-data\\합친 데이터셋\\찐찐찐찐막 전국 데이터.csv')

documents = [df.iloc[idx]['태그'] for idx in range(len(df))]
embedded_docs = embedding_model.encode(documents).tolist()

C:\Users\brain\AppData\Local\Temp\ipykernel_24604\1253391701.py:4: DtypeWarning: Columns (2,55,57,58,59,60,63,64,67,68,69,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\brain\\Desktop\\3차 프로젝트\\GIT\\crawling-data\\합친 데이터셋\\찐찐찐찐막 전국 데이터.csv')


In [ ]:
# 메타 데이터 생성
import re

metadata=[]
for idx in range(len(df)):
    meta_data = {}
    row = df.iloc[idx][df.iloc[idx].notna()]
    cols = list(row.index)   # None이 아닌 컬럼들 == row.index 

    for col in cols:
      if col == '태그':
        meta_data['text'] = row[col]
      elif col == '주소':
        meta_data['region'] = row[col].split()[0]
      elif col == 'place':
        meta_data['place'] = row[col]
    metadata.append(meta_data)


In [3]:
metadata

NameError: name 'metadata' is not defined

In [ ]:
# 컬렉션에 추가
from tqdm import tqdm
collection.add(
    ids = [str(i) for i in tqdm(range(len(documents)))],
    embeddings = [embedded_docs],
    documents = documents,      # 오류나면 빼기
    metadatas = metadata
)

100%|██████████| 19854/19854 [00:00<00:00, 2331225.65it/s]


In [2]:
# 질문 입력 -> 질문 임베딩 -> collection에 전달 -> 상위 결과 5개 뽑기
question = '가족들과 가기 좋은 캠핑장 알려줘'

embedded_question = embedding_model.encode(question).tolist()

top_n = 5
context = collection.query(
    query_embeddings= embedded_question,
    n_results = top_n
)

# 확인용 코드
print(f'Q: {question}')
for i in range(top_n):
    print(context)
    print(f"장소 이름 : {context['metadatas'][i][0]['place']}")
    print(f"주소 : {context['metadatas'][i][0]['주소']}")
    print(f"태그 : {context['metadatas'][i][0]['태그']}")
    print()

NameError: name 'embedding_model' is not defined

In [ ]:
# sllm 모델 생성
from transformers import AutoModel

model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
sllm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 프롬프트 생성 -> sllm 응답 요청 

# 채팅 로그 수정할 수 있는 함수 설정
def modify_chat_log(chatting_log,message):
    if len(chatting_log)% 2 == 0:
        chatting_log.append({"role": "assistant", "content": message})
    else:
        chatting_log.append({'role':'user','content':message})
    return chatting_log

prompt = f'{question} \n 다음 정보를 활용하여 간략하게 10문장 이내로 출력하시오.{context}'

chat_history = [
    {"role": "system", "content": "너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마."}
]

chat_history = modify_chat_log(chat_history,prompt)

inputs = tokenizer.apply_chat_template(chat_history, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to("cuda")
output_ids = sllm.generate(
    **inputs,
    max_length=1024,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
    )
print(tokenizer.batch_decode(output_ids)[0])

---
### Chromadb
- embedding model : 'all-MiniLM-L6-v2'
- Document(page_content = 태그와 짧은 설명 , metadata = 나머지 정보들)

In [ ]:
# pd 생성 후 태그랑 short_info 합쳐서 임베딩하기

df = pd.read_csv('찐막 전국 데이터.csv')

documents = []
for idx in range(len(df)):
    data = ''
    if df.iloc[idx]['태그'] is not None and df.iloc[idx]['short_info'] is not None:
        data = df.iloc[idx]['태그'] + " " + df.iloc[idx]['short_info']
    elif df.iloc[idx]['태그'] is not None and df.iloc[idx]['short_info'] is None:
        data = df.iloc[idx]['태그']
    elif df.iloc[idx]['태그'] is None and df.iloc[idx]['short_info'] is not None:
        data = df.iloc[idx]['short_info']
    else:
        pass
    documents.append(data)

embedded_docs = embedding_model.encode(documents).tolist()

In [ ]:
# 메타 데이터 생성

def remove_nonwords(text):
    if type(text)!= float :
        text = text.replace(':)',"")    # :) 제거
        text = text.replace("※","")
        text = text.replace('-','')
        text = text.replace('\n더보기','')      # \n 더보기, \n 닫기, \n 제거
        text = text.replace('\n닫기','')
        text = text.replace('\r\n',' ')
        text = text.replace('\n',' ')
        text = text.replace('\r', ' ')
        text = re.sub('[^a-zA-Z가-힣\'"· 0-9.,()㎡[0-9]~[0-9]]','',text) # 숫자와 숫자 사이에 있는 ~ 남겨야함
    else:
        pass
    return text     # 제거한 텍스트 반환

metadata=[]
for idx in range(len(df)):
    meta_data = {}
    row = df.iloc[idx][df.iloc[idx].notna()]
    cols = list(row.index)   # None이 아닌 컬럼들 == row.index 

    for col in cols:     
        modified_data = remove_nonwords(row[col])
        meta_data[col] = modified_data
    metadata.append(meta_data)

In [ ]:
# 컬렉션에 추가
collection.add(
    ids = [str(i) for i in range(len(documents))],
    embeddings = [embedded_docs],
    documents = documents,      # 오류나면 빼기
    metadatas = metadata
)

In [ ]:
# 질문 입력 -> 질문 임베딩 -> collection에 전달 -> 상위 결과 5개 뽑기
question = '가족들과 가기 좋은 캠핑장 알려줘'

embedded_question = embedding_model.encode(question).tolist()

top_n = 5
context = collection.query(
    query_embeddings= embedded_question,
    n_results = top_n
)

# 확인용 코드
print(f'Q: {question}')
for i in range(top_n):
    print(context)
    print(f"장소 이름 : {context['metadatas'][i][0]['place']}")
    print(f"주소 : {context['metadatas'][i][0]['주소']}")
    print(f"태그 : {context['metadatas'][i][0]['태그']}")
    print()

In [ ]:
# sllm 모델 생성
from transformers import AutoModel

model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
sllm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 프롬프트 생성 -> sllm 응답 요청 

# 채팅 로그 수정할 수 있는 함수 설정
def modify_chat_log(chatting_log,message):
    if len(chatting_log)% 2 == 0:
        chatting_log.append({"role": "assistant", "content": message})
    else:
        chatting_log.append({'role':'user','content':message})
    return chatting_log

prompt = f'{question} \n 다음 정보를 활용하여 간략하게 10문장 이내로 출력하시오.{context}'

chat_history = [
    {"role": "system", "content": "너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마."}
]

chat_history = modify_chat_log(chat_history,prompt)

inputs = tokenizer.apply_chat_template(chat_history, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to("cuda")
output_ids = sllm.generate(
    **inputs,
    max_length=1024,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
    )
print(tokenizer.batch_decode(output_ids)[0])

----
# Retriever 이용

In [ ]:
# Document로 저장

from langchain.text_splitter import RecursiveCharacterTextSplitter      # 재귀적으로 text splitter  큰 chunk -> 작은 chunk 로 분할할 chunk size 지정
import preprocess_functions as pf

documents = pf.make_tag_page_content('찐막 전국 데이터.csv')    # 태그와 짤막한 설명을 같이 page_content에 넣어주는 경우
documents

In [ ]:
# 임베딩 모델 생성 및 vectorstore 생성
from langchain_chroma.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
vector_store = Chroma.from_documents(documents, embedding_model)

In [ ]:
# 2. Retriever를 활용한 검색

question = '가족들과 가기 좋은 캠핑장 알려줘'

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':10}
)

retriever_result = retriever.batch([question])
retriever_result

In [ ]:
# 프롬프트 템플릿 생성

from langchain_core.prompts import ChatPromptTemplate

chatting_log = [
    ('system', '너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마.'),
    ('user', '''어린이의 질문에 context만을 이용해 답변하세요.
context에서 확인할 수 없는 질문이라면 모른다고 답변해야 합니다.
최종 응답에는 참조한 context에 대한 정보를 추가해 주세요.

     질문: {query}
     context: {context}
''')
]

prompt_tpl = ChatPromptTemplate(chatting_log)
# prompt_tpl.invoke({'query': query, 'context': retriever_result})

In [ ]:
# sllm 모델 생성
from transformers import AutoModel
from transformers.models import AutoModelForCausalLM, AutoTokenizer

model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
sllm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 체인 생성
from langchain_core.output_parsers import StrOutputParser

chain = prompt_tpl | sllm | StrOutputParser()

In [ ]:
retriever_result = retriever.batch([query])
response = chain.invoke({'query': question, 'context': retriever_result})
print(response)